In [4]:
import pathlib

example_input = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

class Node:
    def __init__(self, name):
        self.name = name
        self.children = set()
        self.size = 0
        self.parent = None
    def __str__(self):
        return f"Node({self.name=}, {self.size})"
    def add_child(self, node):
        node.parent = self
        self.children.add(node)
        return node
    def add_size(self, size):
        self.size += size
    def get_size(self):
        total = self.size
        for child in self.children:
            total += child.get_size()
        return total
    def get_subnodes(self):
        subnodes = list(self.children)
        for child in self.children:
            subnodes.extend(child.get_subnodes())
        return subnodes

def p1(data: str):
    root = parse(data)

    total = 0
    for node in root.get_subnodes():
        if node.get_size() < 100000:
            total += node.get_size()
    return total

def parse(data):
    root = Node("/")
    current = root
    for command_results in data.split("$ "):
        if command_results:
            lines = command_results.splitlines()
            if len(lines) > 1:
                command, *ls_results = lines
                assert command == "ls"
                for f in ls_results:
                    if f.split(" ")[0].isnumeric():
                        current.add_size(int(f.split(" ")[0]))
            else:
                command, cd_arg = lines[0].split(" ")
                assert command == "cd"
                if cd_arg == "..":
                    current = current.parent
                elif cd_arg != "/":
                    current = current.add_child(Node(cd_arg))
    return root


assert (res := p1(example_input)) == 95437, f"{res=}"

p1(pathlib.Path("day07.txt").read_text())

1743217

In [9]:
def p2(data: str):
    root = parse(data)
    available = 70000000
    needed = 30000000
    used = root.get_size()
    unused = available - used
    to_delete = needed - unused

    candidates = []
    for node in root.get_subnodes():
        size = node.get_size()
        if size >= to_delete:
            candidates.append(size)
    return min(candidates)


assert (res := p2(example_input)) == 24933642, f"{res=}"

p2(pathlib.Path("day07.txt").read_text())

8319096